# Model Comparison

In this example you will learn how to evaluate various models for RAG with TruLens.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/use_cases/huggingface_models.ipynb)

## Setup
### Install requirements

In [ ]:
!pip install install faiss-cpu unstructured==0.10.12 langchain git@trulens litellm openai trulens_eval flask

### Add API keys
Enter your keys before running the cells below.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_KEY_HERE"
os.environ["HUGGINGFACE_API_KEY"] = "YOUR_KEY_HERE"

In [ ]:
from litellm import completion
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

### Import from TruLens

In [ ]:
# Imports main tools:
from trulens_eval import Feedback
from trulens_eval import OpenAI
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

### Create a Simple RAG Application

This example uses a simple rag application, to demonstrate vectara hallucination for demonstration purposes.

In [ ]:
# inspired by the great example from Joselin James here : https://github.com/truera/trulens/blob/29b7f3152d6058a8a7bbb680fe72368d3f020b53/trulens_eval/examples/expositional/vector-dbs/faiss/langchain_faiss_example.ipynb
from typing import List
from flask import Flask, request, jsonify
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.vectorstores.base import VectorStoreRetriever
import numpy as np

from trulens_eval import feedback
from trulens_eval import Feedback
from trulens_eval import Select
from trulens_eval import Tru

# Create a local FAISS Vector DB based on README.md .
loader = UnstructuredMarkdownLoader("README.md")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)

# Save it.
db.save_local("faiss_index")

class VectorStoreRetrieverWithScore(VectorStoreRetriever):

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        if self.search_type == "similarity":
            docs_and_scores = self.vectorstore.similarity_search_with_relevance_scores(
                query, **self.search_kwargs
            )

            print("From relevant doc in vec store")
            docs = []
            for doc, score in docs_and_scores:
                if score > 0.6:
                    doc.metadata["score"] = score
                    docs.append(doc)
        elif self.search_type == "mmr":
            docs = self.vectorstore.max_marginal_relevance_search(
                query, **self.search_kwargs
            )
        else:
            raise ValueError(f"search_type of {self.search_type} not allowed.")
        return docs

# Create the example app.
class FAISSWithScore(FAISS):

    def as_retriever(self) -> VectorStoreRetrieverWithScore:
        return VectorStoreRetrieverWithScore(
            vectorstore=self,
            search_type="similarity",
            search_kwargs={"k": 4},
        )

class FAISSStore:

    @staticmethod
    def load_vector_store():
        embeddings = OpenAIEmbeddings()
        faiss_store = FAISSWithScore.load_local("faiss_index", embeddings)
        print("Faiss vector DB loaded")
        return faiss_store


# Initialize the FAISS vector store
faiss_store = FAISSStore.load_vector_store()

def retrieve_documents(query: str) -> List[Document]:
    """
    Retrieve relevant documents from the FAISS vector store based on the query.
    """
    retriever = faiss_store.as_retriever()
    documents = retriever._get_relevant_documents(query)
    return documents

def format_documents_for_mixtral(documents: List[Document], user_query: str) -> str:
    """
    Format the retrieved documents and user query for Mixtral input.
    """
    # Combine the text of each document, you might want to adjust the formatting
    combined_docs = " ".join([doc.text for doc in documents])
    # Append the user query to the combined documents
    mixtral_input = f"""Context : 
    {combined_docs} 
    Query :
    {user_query}"""
    return mixtral_input


@app.route('/ask', methods=['POST'])
def ask():
    """
    Endpoint to handle user queries, retrieve documents, and query Mixtral.
    """
    data = request.json
    user_query = data.get("query")
    if not user_query:
        return jsonify({"error": "No query provided"}), 400
    # Retrieve documents based on the user query
    documents = retrieve_documents(user_query)
    # Format the documents and user query for Mixtral
    mixtral_input = format_documents_for_mixtral(documents, user_query)
    # Query Mixtral API
    mixtral_response = query({"inputs": mixtral_input})
    # Return Mixtral's response
    return jsonify(mixtral_response)

# if __name__ == '__main__':
#     app.run(debug=True, port=5000)


## Initialize Feedback Function(s)

In [ ]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a sentiment feedback function using HuggingFace.
f_hallucination = Feedback(hugs.hallucination_evaluator, feedback_mode = FeedbackMode.DEFERRED).on_output()

feedbacks = [f_hallucination]

## Instrument the callable for logging with TruLens

In [ ]:
# Fixing the load_conversational_chain function
def load_conversational_chain(vector_store):
    llm = ask()
    retriever = vector_store.as_retriever()
    chain = ConversationalRetrievalChain.from_llm(
        llm, retriever, return_source_documents=True
    )

    tru = Tru()

    truchain = tru.Chain(chain,feedbacks=feedbacks,with_hugs=True)

    return chain, truchain

In [ ]:
prompts = [
    "Describe the implications of widespread adoption of autonomous vehicles on urban infrastructure.",
    "Write a short story about a world where humans have developed telepathic communication.",
    "Debate the ethical considerations of using CRISPR technology to genetically modify humans.",
    "Compose a poem that captures the essence of a dystopian future ruled by artificial intelligence.",
    "Explain the concept of the multiverse theory and its relevance to theoretical physics.",
    "Provide a detailed plan for a sustainable colony on Mars, addressing food, energy, and habitat.",
    "Discuss the potential benefits and drawbacks of a universal basic income policy.",
    "Imagine a dialogue between two AI entities discussing the meaning of consciousness.",
    "Elaborate on the impact of quantum computing on cryptography and data security.",
    "Create a persuasive argument for or against the colonization of other planets as a solution to overpopulation on Earth."
]

In [ ]:
tru.run_dashboard()

In [ ]:
with mistral7b_recorder as recording:
    for prompt in prompts:
        mistral7b_recorder.app(prompt_input)

## Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all